<a href="https://colab.research.google.com/github/hatewxb/ml-projects/blob/main/ML_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание 1

In [16]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Загрузка данных
file_path = "1.xlsx"
data = pd.read_excel(file_path, sheet_name="\u0414\u0430\u043d\u043d\u044b\u0435")

# Независимые переменные
X = data[["Wages", "WorkAge", "SecEdu"]]
# Целевая переменная
y = data["SecStudent"]

# Добавляем константу для регрессии
X_with_const = sm.add_constant(X)

# Строим модель множественной линейной регрессии
model = sm.OLS(y, X_with_const).fit()

# Проверка мультиколлинеарности
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Результаты задачи 1
print("=== Задача 1 ===")
print("Множественная линейная регрессия:")
print(model.summary())
print("\nПроверка мультиколлинеарности (VIF):")
print(vif_data)

# Дополнительный анализ значимости факторов
X_reduced = data[["Wages"]]  # Убираем незначимые переменные
X_reduced_with_const = sm.add_constant(X_reduced)

# Линейная регрессия с одним фактором
model_reduced = sm.OLS(y, X_reduced_with_const).fit()
print("\nМодель с одним фактором (Wages):")
print(model_reduced.summary())

=== Задача 1 ===
Множественная линейная регрессия:
                            OLS Regression Results                            
Dep. Variable:             SecStudent   R-squared:                       0.099
Model:                            OLS   Adj. R-squared:                  0.065
Method:                 Least Squares   F-statistic:                     2.958
Date:                Fri, 24 Jan 2025   Prob (F-statistic):             0.0372
Time:                        11:42:42   Log-Likelihood:                -408.76
No. Observations:                  85   AIC:                             825.5
Df Residuals:                      81   BIC:                             835.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from scipy.stats import pearsonr, f_oneway

# Загрузка данных
file_path = "1.xlsx"  # Укажите путь к файлу
main_data = pd.read_excel(file_path, sheet_name="Данные")

# 1. Линейная регрессия с проверкой значимости факторов
# Логарифмическое преобразование переменных
main_data['log_Wages'] = np.log(main_data['Wages'])
main_data['log_WorkAge'] = np.log(main_data['WorkAge'])
main_data['log_Popul'] = np.log(main_data['Popul'])

# Подготовка данных
X_transformed = main_data[['log_Wages', 'log_WorkAge', 'SecEdu']].values
y = main_data['SecStudent'].values

# Добавление константы
X_with_const = sm.add_constant(X_transformed)

# Линейная регрессия
model = sm.OLS(y, X_with_const).fit()
regression_summary = model.summary()

# 2. Корреляционная матрица (Пирсона)
correlation_matrix = np.corrcoef(X_transformed.T, y)[-1, :-1]

# 3. Проверка гипотез через ANOVA
# Разделение регионов на группы по зарплатам (квантильное разбиение)
main_data['Wages_Group'] = pd.qcut(main_data['Wages'], 3, labels=["Low", "Medium", "High"])
groups = [
    main_data.loc[main_data['Wages_Group'] == group, 'SecStudent'].dropna()
    for group in ["Low", "Medium", "High"]
]
anova_test = f_oneway(*groups)

# 4. Нормализация данных и повторная линейная регрессия
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X_transformed)

# Добавление константы
X_normalized_with_const = sm.add_constant(X_normalized)

# Линейная регрессия на нормализованных данных
model_normalized = sm.OLS(y, X_normalized_with_const).fit()
normalized_summary = model_normalized.summary()

# Вывод результатов
print("=== Линейная регрессия (оригинальные данные) ===")
print(regression_summary)

print("\n=== Корреляция факторов с численностью студентов СПО ===")
print("Корреляция с SecStudent:", correlation_matrix)

print("\n=== ANOVA-тест ===")
print(f"F-статистика: {anova_test.statistic}, p-значение: {anova_test.pvalue}")

print("\n=== Линейная регрессия (нормализованные данные) ===")
print(normalized_summary)


=== Линейная регрессия (оригинальные данные) ===
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.074
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     2.159
Date:                Fri, 24 Jan 2025   Prob (F-statistic):             0.0992
Time:                        09:46:45   Log-Likelihood:                -409.91
No. Observations:                  85   AIC:                             827.8
Df Residuals:                      81   BIC:                             837.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
con

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Загрузка данных
file_path = "path_to_your_file.xlsx"
data = pd.read_excel('1.xlsx', sheet_name="Данные")

# ЗАДАЧА 1: Анализ зависимости численности студентов СПО
# Выбираем независимые переменные
X = data[['Wages', 'WorkAge', 'SecEdu',  ]]
y = data['SecStudent']

# Добавляем константу для регрессии
X_with_const = sm.add_constant(X)

# Строим модель множественной линейной регрессии
model = sm.OLS(y, X_with_const).fit()

# Проверка мультиколлинеарности
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Вывод результатов
print("Множественная линейная регрессия:")
print(model.summary())
print("\nПроверка мультиколлинеарности (VIF):")
print(vif_data)

Множественная линейная регрессия:
                            OLS Regression Results                            
Dep. Variable:             SecStudent   R-squared:                       0.099
Model:                            OLS   Adj. R-squared:                  0.065
Method:                 Least Squares   F-statistic:                     2.958
Date:                Fri, 24 Jan 2025   Prob (F-statistic):             0.0372
Time:                        08:56:35   Log-Likelihood:                -408.76
No. Observations:                  85   AIC:                             825.5
Df Residuals:                      81   BIC:                             835.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        201.0

 R-squared:                       0.099 указывает, что модель объясняет всего 9.9% вариации в численности студентов СПО, что является низким значением.


Wages/P>|t| : 0.011 средняя зарплата оказалась статистически значимой с Wages/coef: -0.0004  что означает, что с увеличением зарплаты численность студентов СПО снижается.

WorkAge и SecEdu не являются значимыми предикторами  так как p-value > 0.05

Задание 2

Балансировка

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# Загрузка данных
file_path = "path_to_your_file.xlsx"
data = pd.read_excel('1.xlsx', sheet_name="Данные")

# 1. Oversampling с помощью SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Проверка распределения классов после SMOTE
smote_distribution = y_train_smote.value_counts()

# 2. Undersampling для большинства класса
undersampler = RandomUnderSampler(random_state=42)
X_train_under, y_train_under = undersampler.fit_resample(X_train, y_train)

# Проверка распределения классов после undersampling
undersample_distribution = y_train_under.value_counts()

# 3. Random Forest с учетом дисбаланса классов
rf_balanced = RandomForestClassifier(random_state=42, class_weight='balanced')
rf_balanced.fit(X_train, y_train)
rf_balanced_pred = rf_balanced.predict(X_test)
rf_balanced_report = classification_report(y_test, rf_balanced_pred)
rf_balanced_accuracy = accuracy_score(y_test, rf_balanced_pred)

# Собираем результаты
results_balancing = {
    "SMOTE Class Distribution": smote_distribution,
    "Undersampling Class Distribution": undersample_distribution,
    "Random Forest Balanced Report": rf_balanced_report,
    "Random Forest Balanced Accuracy": rf_balanced_accuracy,
}

results_balancing


{'SMOTE Class Distribution': HighEduRegion
 0    31
 1    31
 Name: count, dtype: int64,
 'Undersampling Class Distribution': HighEduRegion
 0    28
 1    28
 Name: count, dtype: int64,
 'Random Forest Balanced Report': '              precision    recall  f1-score   support\n\n           0       0.92      1.00      0.96        12\n           1       1.00      0.93      0.96        14\n\n    accuracy                           0.96        26\n   macro avg       0.96      0.96      0.96        26\nweighted avg       0.96      0.96      0.96        26\n',
 'Random Forest Balanced Accuracy': 0.9615384615384616}

Выполнение

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Загрузка данных
file_path = "path_to_your_file.xlsx"
data = pd.read_excel('1.xlsx', sheet_name="Данные")

# ЗАДАЧА 2: Сравнение регионов (вместо федеральных городов добавляем другую категорию)
# Добавляем новую категорию (например, регионы с высоким уровнем образования)
data['HighEduRegion'] = (data['HighEdu'] > data['HighEdu'].median()).astype(int)

# Подготовка данных
X_features = data[['Wages', 'HighEdu', 'SecEdu', 'SecStudent', 'HighStudent', 'Popul', 'WorkAge']]
y_target = data['HighEduRegion']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size=0.3, random_state=42)

# Модели для классификации
# Логистическая регрессия
logreg = LogisticRegression(random_state=42)
logreg.fit(X_train, y_train)
logreg_pred = logreg.predict(X_test)

# Случайный лес
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

# Кластеризация (KMeans)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_features)
kmeans = KMeans(n_clusters=2, random_state=42)
clusters = kmeans.fit_predict(X_scaled)

# PCA для визуализации
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Вывод результатов
print("\nЛогистическая регрессия:")
print(classification_report(y_test, logreg_pred))
print(f"Accuracy: {accuracy_score(y_test, logreg_pred)}")

print("\nСлучайный лес:")
print(classification_report(y_test, rf_pred))
print(f"Accuracy: {accuracy_score(y_test, rf_pred)}")

print("\nKMeans кластеры:")
print(clusters[:10])  # Пример первых 10 кластеров

print("\nPCA объяснённая дисперсия:")
print(pca.explained_variance_ratio_)



/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



Логистическая регрессия:
              precision    recall  f1-score   support

           0       0.71      1.00      0.83        12
           1       1.00      0.64      0.78        14

    accuracy                           0.81        26
   macro avg       0.85      0.82      0.81        26
weighted avg       0.86      0.81      0.80        26

Accuracy: 0.8076923076923077

Случайный лес:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96        12
           1       1.00      0.93      0.96        14

    accuracy                           0.96        26
   macro avg       0.96      0.96      0.96        26
weighted avg       0.96      0.96      0.96        26

Accuracy: 0.9615384615384616

KMeans кластеры:
[1 1 1 1 1 1 1 1 1 1]

PCA объяснённая дисперсия:
[0.37055205 0.22710113]


In [20]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Подготовка данных
federal_cities = ["г. Москва", "г. Санкт-Петербург", "г. Севастополь"]
data["IsFederalCity"] = data["Region"].isin(federal_cities).astype(int)

# Выбираем независимые переменные
X = data[["Wages", "HighEdu", "SecEdu", "SecStudent", "HighStudent", "Popul", "WorkAge"]]
y = data["IsFederalCity"]

# Масштабируем данные
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Разделение данных на обучающую и тестовую выборки с учетом стратификации
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Проверяем распределение классов
print("Распределение классов в обучающей выборке:")
print(y_train.value_counts())

# Если классы всё ещё несбалансированы, уменьшаем k_neighbors в SMOTE
try:
    smote = SMOTE(random_state=42, k_neighbors=min(1, y_train.value_counts()[1] - 1))
    X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)
except ValueError:
    print("SMOTE не может быть применён из-за недостаточного числа наблюдений меньшинства. Используем ручное дублирование.")
    minority_class = X_train[y_train == 1]
    majority_class = X_train[y_train == 0]
    minority_labels = y_train[y_train == 1]
    majority_labels = y_train[y_train == 0]

    X_train_balanced = pd.concat([majority_class, minority_class.sample(len(majority_class), replace=True)])
    y_train_balanced = pd.concat([majority_labels, minority_labels.sample(len(majority_labels), replace=True)])

# Логистическая регрессия
logreg = LogisticRegression(random_state=42)
logreg.fit(X_train_balanced, y_train_balanced)
logreg_pred = logreg.predict(X_test)

print("\n=== Задача 2: Логистическая регрессия ===")
print("Accuracy:", accuracy_score(y_test, logreg_pred))
print(classification_report(y_test, logreg_pred))

# Случайный лес
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_balanced, y_train_balanced)
rf_pred = rf.predict(X_test)

print("\n=== Задача 2: Случайный лес ===")
print("Accuracy:", accuracy_score(y_test, rf_pred))
print(classification_report(y_test, rf_pred))

# KMeans кластеризация
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans_clusters = kmeans.fit_predict(X_scaled)
print("\n=== Задача 2: KMeans ===")
print("Кластеры первых 10 регионов:", kmeans_clusters[:10])

# PCA для визуализации
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)
print("\n=== Задача 2: PCA ===")
print("Доля объясненной дисперсии главными компонентами:", pca.explained_variance_ratio_)


Распределение классов в обучающей выборке:
IsFederalCity
0    57
1     2
Name: count, dtype: int64


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



=== Задача 2: Логистическая регрессия ===
Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        25
           1       1.00      1.00      1.00         1

    accuracy                           1.00        26
   macro avg       1.00      1.00      1.00        26
weighted avg       1.00      1.00      1.00        26


=== Задача 2: Случайный лес ===
Accuracy: 0.9615384615384616
              precision    recall  f1-score   support

           0       1.00      0.96      0.98        25
           1       0.50      1.00      0.67         1

    accuracy                           0.96        26
   macro avg       0.75      0.98      0.82        26
weighted avg       0.98      0.96      0.97        26


=== Задача 2: KMeans ===
Кластеры первых 10 регионов: [1 1 1 1 1 1 1 1 1 1]

=== Задача 2: PCA ===
Доля объясненной дисперсии главными компонентами: [0.37055205 0.22710113]
